# KoBERT finetuning

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

# Import packages

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split, SubsetRandomSampler, ConcatDataset
from sklearn.model_selection import KFold
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm
import tensorboard
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model

In [ ]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
## CPU
#device = torch.device("cpu")

## GPU
device = torch.device("cuda:0")

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue May 17 23:08:24 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Fix Seed 
For reproductibility, fix seed

In [ ]:
# Seed everything for reproductibility
def seed_everything(seed: int = 42):
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore

In [ ]:
SEED = 2022
seed_everything(SEED)

# Dataset


In [ ]:
import pandas as pd
#불러올 파일의 경로를 filename 변수에 저장
filename = '/content/drive/MyDrive/total_data.csv'

#pandas read_csv로 불러오기
data = pd.read_csv(filename, names=['ID', "data", "label"], encoding='cp949', header=None)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
# 불필요한 row, column 제거
data = data.drop(['ID'], axis=1)
data = data.drop([0], axis=0)

In [ ]:
print("전체 데이터 개수")
print("Label 0: ", data[data.label==0].shape[0])
print("Label 1: ", data[data.label==1].shape[0])

전체 데이터 개수
Label 0:  2415122
Label 1:  1484108


In [ ]:
#data = data.sample(frac=0.0001)

In [ ]:
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data, test_size=0.2)

In [ ]:
print("Train Dataset 데이터 개수")
print("Train - Label 0: ", dataset_train[dataset_train.label==0].shape[0])
print("Train - Label 1: ", dataset_train[dataset_train.label==1].shape[0])
print("Test Dataset 데이터 개수")
print("Test - Label 0: ", dataset_test[dataset_test.label==0].shape[0])
print("Test - Label 1: ", dataset_test[dataset_test.label==1].shape[0])

Train Dataset 데이터 개수
Train - Label 0:  1932431
Train - Label 1:  1186894
Test Dataset 데이터 개수
Test - Label 0:  482691
Test - Label 1:  297214


In [ ]:
bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [ ]:
# 토크나이저
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        dataset_len = dataset.shape[0]

        self.sentences = [transform([str(dataset.iloc[i][0])]) for i in range(dataset_len)]
        self.labels = [np.int32(dataset.iloc[i][1]) for i in range(dataset_len)]
        
        #self.sentences = [transform(dataset['data'].tolist())]
        #self.labels = [np.int32(dataset['label'].tolist())]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [ ]:
## Setting parameters
max_len = 64
batch_size = 128
K = 3

In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)

In [ ]:
# Cross Validation
splits = KFold(n_splits=K, shuffle=True, random_state=SEED)

# Model

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        ㅋ`
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        else:
            out = pooler
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

# Train

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
# setting hyper params
warmup_ratio = 0.1
num_epochs = 3
max_grad_norm = 1
log_interval = 100
save_interval = 1000
learning_rate =  5e-5

In [ ]:
# optimizer
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)

# loss function
loss_fn = nn.CrossEntropyLoss()

# cosine warmup scheduler
t_total = len(dataset_train)
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    acc = accuracy_score(max_indices.numpy(),Y)
    f1 = f1_score(max_indices.numpy(),Y)
    recall = recall_score(max_indices.numpy(),Y)
    precision = precision_score(max_indices.numpy(),Y)
    return acc, f1, recall, precision

In [ ]:
# directory to save .pth file
MODEL_PATH = '/content/drive/MyDrive/model_ckpt/'

In [ ]:
# If continue - 연속해서 학습시킬때만 사용
model.load_state_dict(torch.load(MODEL_PATH+'fold2epoch3.pth')['model'])
continue_fold = 2
continue_epoch = 0
continue_batch = 0

In [ ]:
def train_epoch(fold, epoch, model, device, dataloader, loss_fn, optimizer, continue_batch):
    train_acc = 0
    train_f1 = 0
    train_recall = 0
    train_precision = 0

    # log file
    #LOG_PATH = '/content/drive/MyDrive/logging.txt'
    #f = open(LOG_PATH, 'a')

    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(dataloader), total=len(dataloader)):
        # for continue
        if batch_id < continue_batch:
          continue
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        acc, f1 ,recall, precision= calc_accuracy(out.cpu().detach(), label.cpu().detach())
        train_acc += acc
        train_f1 += f1
        train_recall += recall
        train_precision += precision
        if batch_id % log_interval == 0:
            log_line = "Fold {} Epoch {} Batch {} Loss {} Acc {} F1 {} Recall {} Precision {}".format(fold+1, epoch+1, batch_id+1, loss.data.cpu().numpy(), train_acc/(batch_id-continue_batch+1), train_f1/(batch_id-continue_batch+1), train_recall/(batch_id-continue_batch+1), train_precision/(batch_id-continue_batch+1))
            print(log_line)
            #f.write(log_line+"\n")
        if batch_id % save_interval == 0:
            PATH = MODEL_PATH+f'latest.pth'
            torch.save({
              'epoch': epoch,
              'model': model.state_dict()
            }, PATH)
            print("Saved model at "+PATH)

    size = len(dataloader)
    print("epoch {} finish".format(epoch))
    #f.close()
    return loss.data.cpu().numpy(), train_acc/size, train_f1/size, train_recall/size, train_precision/size

In [ ]:
def valid_epoch(fold, epoch, model, device, dataloader, loss_fn):
    valid_acc = 0.0
    valid_f1 = 0.0
    valid_recall = 0.0
    valid_precision = 0.0

    # log file
    #LOG_PATH = '/content/drive/MyDrive/logging.txt'
    #f = open(LOG_PATH, 'a')

    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(dataloader), total=len(dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        acc, f1 ,recall, precision= calc_accuracy(out.cpu().detach(), label.cpu().detach())
        valid_acc += acc
        valid_f1 += f1
        valid_recall += recall
        valid_precision += precision
    size = len(dataloader)
    logline = "Valid Result - Fold {} Epoch {} batch {} Loss {} Acc {} F1 {} Recall {} Precision {}".format(fold+1, epoch+1, batch_id+1, loss.data.cpu().numpy(), valid_acc/size, valid_f1/size, valid_recall/size, valid_precision/size)
    print(logline)
    #f.write(logline+"\n")
    #f.close()
    return loss.data.cpu().numpy(), valid_acc/size, valid_f1/size, valid_recall/size, valid_precision/size

In [ ]:
# log file
"""WARNING!!!"""
"""
You should make empty file 'logging.txt' at google drive before training. (첫 실행시에만)
"""

for fold, (train_idx,val_idx) in enumerate(splits.split(np.arange(len(data_train)))):# log file
 
    if fold < continue_fold:
      continue
    # Get sampler and DataLoader
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(val_idx)
    train_loader = DataLoader(data_train, batch_size=batch_size, sampler=train_sampler)
    valid_loader = DataLoader(data_train, batch_size=batch_size, sampler=valid_sampler)
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)

    for epoch in range(continue_epoch, num_epochs):
        
        train_loss, train_acc, train_f1, train_recall, train_precision = train_epoch(fold, epoch, model, device, train_loader,loss_fn, optimizer, continue_batch)
        valid_loss, valid_acc, valid_f1, valid_recall, valid_precision = valid_epoch(fold, epoch, model, device, valid_loader,loss_fn)

        continue_batch = 0 # reset continue batch

        PATH = MODEL_PATH+f'fold{fold+1}epoch{epoch+1}.pth'
        torch.save({
              'epoch': epoch+1,
              'model': model.state_dict(),
              'Fold': fold+1
            }, PATH)
        print("Saved model at "+PATH)

  0%|          | 0/17340 [00:00<?, ?it/s]

Fold 3 Epoch 1 Batch 1 Loss 0.2542818486690521 Acc 0.921875 F1 0.8979591836734694 Recall 0.88 Precision 0.9166666666666666
Saved model at /content/drive/MyDrive/model_ckpt/latest.pth
Fold 3 Epoch 1 Batch 101 Loss 0.20317396521568298 Acc 0.9410581683168316 F1 0.9245906596759945 Recall 0.911415245618008 Precision 0.9392768732889158
Fold 3 Epoch 1 Batch 201 Loss 0.09940273314714432 Acc 0.9430970149253731 F1 0.9265800894566745 Recall 0.9139017596698306 Precision 0.9407635495354683
Fold 3 Epoch 1 Batch 301 Loss 0.13319334387779236 Acc 0.9426650747508306 F1 0.9262202691386767 Recall 0.913735039237476 Precision 0.9403406252822455
Fold 3 Epoch 1 Batch 401 Loss 0.1386096030473709 Acc 0.9424875311720698 F1 0.9258503930463751 Recall 0.9130016577897714 Precision 0.9403889158885478
Fold 3 Epoch 1 Batch 501 Loss 0.09748255461454391 Acc 0.9420533932135728 F1 0.9251944892282817 Recall 0.9120104928863529 Precision 0.9401212470578111
Fold 3 Epoch 1 Batch 601 Loss 0.11502792686223984 Acc 0.94225769550748

# Test

In [ ]:
def model_eval(model, device, dataloader, loss_fn):
    valid_acc = 0.0
    valid_f1 = 0.0
    valid_recall = 0.0
    valid_precision = 0.0
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(dataloader), total=len(dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        acc, f1 ,recall, precision = calc_accuracy(out.cpu().detach(), label.cpu().detach())
        valid_acc += acc
        valid_f1 += f1
        valid_recall += recall
        valid_precision += precision
    size = len(dataloader)
    print("Test Result")
    print("Loss {} Accuracy {} F1 {} Recall {} Precision {}".format(loss.data.cpu().numpy(), valid_acc/size, valid_f1/size, valid_recall/size, valid_precision/size))
    return loss.data.cpu().numpy(), valid_acc/size, valid_f1/size, valid_recall/size, valid_precision/size

In [ ]:
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [ ]:
test_loader = DataLoader(data_test, batch_size=batch_size)
model.load_state_dict(torch.load(MODEL_PATH+'latest.pth')['model'])
test_loss, test_acc, test_f1, test_recall, test_precision = model_eval(model, device, test_loader, loss_fn)

## Visualize result (수정필요)



In [ ]:
train_acc = []
test_acc = []
train_f1 = []
test_f1 = []
train_loss = []
test_loss = []
for f in range(1, 1+K):
     train_loss.extend(foldperf['fold{}'.format(f)]['train_loss'])
     test_loss.extend(foldperf['fold{}'.format(f)]['test_loss'])

     train_acc.extend(foldperf['fold{}'.format(f)]['train_acc'])
     test_acc.extend(foldperf['fold{}'.format(f)]['test_acc'])

     train_f1.extend(foldperf['fold{}'.format(f)]['train_f1'])
     test_f1.extend(foldperf['fold{}'.format(f)]['test_f1'])

In [ ]:
def convert_np_to_float(np):
  return np.item(0)
  
train_loss = list(map(convert_np_to_float, train_loss))
test_loss = list(map(convert_np_to_float, test_loss))

In [ ]:
def convert_tensor_to_float(t):
  return t.item()

train_f1 = list(map(convert_tensor_to_float, train_f1))
test_f1 = list(map(convert_tensor_to_float, test_f1))

In [ ]:
from matplotlib import pyplot as plt
X = [1, 2, 3, 4, 5, 6,7,8,9]
plt.plot(X, train_loss)
plt.plot(X, test_loss)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Loss")
plt.legend(['Train', 'Test'])
plt.show()

In [ ]:
plt.plot(X, train_acc)
plt.plot(X, test_acc)
plt.xlabel("Epoch")
plt.ylabel("Acc")
plt.title("Acc")
plt.legend(['Train', 'Test'])
plt.show()

In [ ]:
plt.plot(X, train_f1)
plt.plot(X, test_f1)
plt.xlabel("Epoch")
plt.ylabel("F1")
plt.title("F1-Score")
plt.legend(['Train', 'Test'])
plt.show()